In [15]:
import numpy as np
import pandas as pd
from scipy.stats import randint

x=pd.read_csv('processed_scaled.csv',index_col=0)
y=pd.read_csv('grad_rate.csv', header=None)[1].values

In [18]:
x.shape

(121, 193)

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor(criterion='mse', n_jobs=-1, random_state=5)

rf_params = {'n_estimators': randint(5,100), 
             'max_features':randint(10, 50),
             'max_depth': randint(5, 10), 
             'min_samples_split': randint(5, 20)}

rand_search = RandomizedSearchCV(estimator=rf, param_distributions=rf_params, cv=5, n_jobs=-1, n_iter=50)
rand_search.fit(x, y)
print rand_search.best_score_
print rand_search.best_params_

0.8706628482472929
{'max_features': 44, 'min_samples_split': 7, 'n_estimators': 65, 'max_depth': 8}


In [17]:
x[:20]

,Student_Count_Total,Average_ACT_School,College_Enrollment_Rate_School,Graduation_Rate_School,Growth_ACT_Grade_11_Pct,Attainment_ACT_Grade_11_Pct,Freshmen_On_Track_School_Pct_Year_2,College_Persistence_School_Pct_Year_2,lib_cnt,PRED_NON_ENG_PERCENT,...,Neighborhood Name_WEST ELSDON,Neighborhood Name_WEST ENGLEWOOD,Neighborhood Name_WEST LAWN,Neighborhood Name_WEST RIDGE,Neighborhood Name_WEST TOWN,Neighborhood Name_WOODLAWN,PRED_NON_ENG_LANG_AFRICAN LANGUAGES,PRED_NON_ENG_LANG_CHINESE,PRED_NON_ENG_LANG_POLISH,PRED_NON_ENG_LANG_SPANISH
609764,0.385802,0.206061,0.541756,0.827198,0.223881,0.166667,0.691667,0.513274,0.545455,0.888889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
400054,0.216376,0.533333,0.917559,0.923313,0.940299,0.739583,0.793333,0.607670,0.363636,0.844444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
609726,0.208891,0.569697,0.911135,0.955010,0.462687,0.802083,0.906667,0.861357,0.272727,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
400094,0.110002,0.266667,0.778373,0.714724,0.656716,0.260417,0.860000,0.587021,0.181818,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
609755,0.480381,0.896970,0.912206,0.983640,0.641791,1.000000,0.965000,0.941003,0.636364,0.044444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
609756,0.654570,0.260606,0.704497,0.777096,0.208955,0.250000,0.528333,0.665192,0.454545,0.711111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
610518,0.041960,0.090909,0.428266,0.724949,0.238806,0.041667,0.260000,0.265487,0.363636,0.066667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
400077,0.135405,0.248485,0.860814,0.780164,0.507463,0.229167,0.700000,0.811209,0.181818,0.022222,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
400056,0.098889,0.327273,0.823340,0.842536,0.611940,0.375000,0.421667,0.609145,0.272727,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
610394,0.046949,0.151515,0.700214,0.759714,0.179104,0.093750,0.751667,0.442478,0.363636,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
